## Importação das LIBS

In [7]:
import pyspark
import os
from delta import *
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
import pandas as pd
from configs import config_env_test
from functions import functions as func
import logging


builder = pyspark.sql.SparkSession.builder.appName("LocalDelta") \
    .master("local") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
print(spark.version)


3.4.1


In [ ]:
## Mapping

In [8]:
prefix_layer_name_source = config_env_test.prefix_layer_name['1'] #bronze layer
hdfs_source = config_env_test.hdfs_path['bronze']

prefix_layer_name_target = config_env_test.prefix_layer_name['2'] #silver layer
hdfs_target = config_env_test.hdfs_path['silver']

In [ ]:
## Refinement silver tables

In [9]:
# Configure the logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Log the start of ingestions
logging.info("Starting silver refinement...")

for key, value in config_env_test.tables_postgres_adventureworks.items():
    table = value
    table_name = func.convert_table_name(table) 
    
    try:
        df = spark.read.format("delta").load(f'{hdfs_source}{prefix_layer_name_source}{table_name}')                 
        df.write.format("delta").mode("overwrite").save(f'{hdfs_target}{prefix_layer_name_target}{table_name}')        
        logging.info(f"Table {table_name} successfully processed and saved to HDFS: {hdfs_target}{prefix_layer_name}{table_name}")
        
    except Exception as e:
        logging.error(f"Error processing table {table}: {str(e)}")

logging.info("Silver refinemente completed!")

